In [1]:
import models.resnet as ResNet
import torch
import torchvision
from torchvision import transforms
import utils
import json
import numpy as np
import imdirect
import numpy as n
import torch
import gc
from PIL import Image
from matplotlib import pyplot as plt
from torch.autograd import Variable
import cv2
from torch import nn
from pathlib import Path
import tqdm
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

# !!!!  code from https://github.com/cydonia999/VGGFace2-pytorch/  !!!!

In [4]:
DOWNLOADED_WEIGHTS_PATH = '/home/tva/edu/sk/rwaa/resnet50_ft_weight.pkl'

# To train from vggface2 pretrained model you need to download weights from the github repository
net = ResNet.resnet50(num_classes=8631, include_top=True)
utils.load_state_dict(net, DOWNLOADED_WEIGHTS_PATH) # download weights from github above
net.fc = nn.Linear(512 * 4, 20)

In [5]:
# we want to train only last layer
def get_parameters(model, bias=False):
    for k, m in model._modules.items():
        if k == "fc" and isinstance(m, nn.Linear):
            if bias:
                yield m.bias
            else:
                yield m.weight

In [6]:
class FaceDataset(Dataset):
    def __init__(self, root_path, train_test_path, mode='train', transform=None):
        self.markup = []
        self.root_path = root_path
        self.transform = transform
        self.mean_bgr = np.array([91.4953, 103.8827, 131.0912])  # values taken from source repo. Net was pretrained with them
        
        with open(train_test_path, 'r') as f:
            self.train_test_split = json.load(f)
        
        self.ordered_classes = sorted(list(self.train_test_split.keys()))
        self.class_marks = {name: i for i, name in enumerate(self.ordered_classes)}
        
        for i, person in enumerate(self.ordered_classes):
            self.markup.extend([
                (str((Path(self.root_path) / person / img_name).absolute()), i)
                for img_name in self.train_test_split[person][mode]
            ])
        
    def __getitem__(self, index):
        img_path, label = self.markup[index]
        img = Image.open(img_path)
        img = torchvision.transforms.Resize((224, 224))(img)
        
        if self.transform:
            img = self.transform(img)
        
        img = np.array(img)
        img = img[:, :, ::-1]  # RGB -> BGR
        img = img.astype(np.float32)
        img -= self.mean_bgr
        img = img.transpose(2, 0, 1)  # C x H x W        
        img = torch.from_numpy(img).float()

        
        return img, label
    
    
    def __len__(self,):
        return len(self.markup)

In [7]:
# Some training time augmentations
train_transform = transforms.Compose([
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((-5, 5)),
    
])

In [8]:
ALIGNED_DATASET_PATH = '/home/tva/edu/sk/aligned_dev20/'
TRAIN_TEST_SPLIT_PATH = '/home/tva/edu/sk/train_test_markup.json'

train_dataset = FaceDataset(root_path=ALIGNED_DATASET_PATH,
                            train_test_path=TRAIN_TEST_SPLIT_PATH,
                            mode='train',
                            transform=train_transform)

test_dataset = FaceDataset(root_path=ALIGNED_DATASET_PATH,
                            train_test_path=TRAIN_TEST_SPLIT_PATH,
                            mode='test',
                            transform=None)

train_loader = DataLoader(train_dataset, batch_size=40, shuffle=True, num_workers=3)
val_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=3)

In [9]:
# We need to specify num_classes=8631 to load pretrained weights, because the model was trained on that amount of classes
# Just replace the last FullyConnected layer with a new one
net = ResNet.resnet50(num_classes=8631, include_top=True)
utils.load_state_dict(net, '/home/tva/edu/sk/rwaa/resnet50_ft_weight.pkl') # download weights from github above
net.fc = nn.Linear(512 * 4, 20)


## Or you can just load our own pretrained weights
net.load_state_dict(torch.load('finetuned.pth'))


# device can be either 'cuda' or 'cpu'. Define it once and use .to(device) everywhere 
device = torch.device('cuda')
net.to(device)

lf = nn.CrossEntropyLoss()
lf = lf.to(device)

In [10]:
import types

args = types.SimpleNamespace()
args.print_freq = 100
args.lr = 0.1
args.lr_update_freq = 1000  # update lr every ... iteration

In [11]:
optim = torch.optim.Adam(
            [
                {'params': get_parameters(net, bias=False)},
                {'params': get_parameters(net, bias=True), 'lr': args.lr * 2, 'weight_decay': 0},
            ],
            lr=args.lr,)

# optim = torch.optim.Adam(net.parameters(),
#             lr=args.lr,)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optim, args.lr_update_freq, gamma=0.1)

In [12]:


# A function to evaluate performance on the validation set 
def validate(training=True):
    best_top1 = 0.0
    best_top5 = 0.0
    batch_time = utils.AverageMeter()
    losses = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()

    net.eval()
    
    iteration = 0
    for batch_idx, (imgs, target) in tqdm.tqdm(
            enumerate(val_loader), total=len(val_loader),
            desc='Valid iteration={}'.format(iteration), ncols=80, leave=False):

        gc.collect()
        iteration += 1
        imgs, target = imgs.to(device), target.to(device)
            
        output = net(imgs)
        loss = lf(output, target)

        if np.isnan(float(loss.item())):
            raise ValueError('loss is nan while validating')

        # measure accuracy and record loss
        prec1, prec5 = utils.accuracy(output.data, target.data, topk=(1, 5))
        losses.update(loss.item(), imgs.size(0))
        top1.update(prec1[0], imgs.size(0))
        top5.update(prec5[0], imgs.size(0))

    if training == True:
        is_best = top1.avg > best_top1
        best_top1 = max(top1.avg, best_top1)
        best_top5 = max(top5.avg, best_top5)

        log_str = 'Test_summary: [{0}/{1}/{top1.count:}] iter: {iteration:}\t' \
              'BestPrec@1: {best_top1:.3f}\tBestPrec@5: {best_top5:.3f}\t' \
              'Loss: {loss.avg:.4f}\t' \
              'Prec@1: {top1.avg:.3f}\tPrec@5: {top5.avg:.3f}\t'.format(
            batch_idx, len(val_loader), iteration=iteration,
            best_top1=best_top1, best_top5=best_top5,
           loss=losses, top1=top1, top5=top5)
        print(log_str)

        if training:
            net.train()

                

In [13]:
batch_time = utils.AverageMeter()
losses = utils.AverageMeter()
top1 = utils.AverageMeter()  # top1 precision
top5 = utils.AverageMeter()  # top5 precision


# train loop        
for epoch in range(1000000):
    iteration = 0
    for batch_idx, (imgs, target) in tqdm.tqdm(
                    enumerate(train_loader), total=len(train_loader),
                    desc='Train epoch={}, iter={}'.format(epoch, iteration), ncols=80, leave=False):
        iteration = batch_idx + epoch * len(train_loader)

        imgs = imgs.to(device)
        target = target.to(device)

        output = net(imgs)
        loss = lf(output, target)

        if np.isnan(float(loss.item())):
            raise ValueError('loss is nan while training')

        # measure accuracy and record loss
        prec1, prec5 = utils.accuracy(output.data, target.data, topk=(1, 5))
        losses.update(loss.item(), imgs.size(0))
        top1.update(prec1[0], imgs.size(0))
        top5.update(prec5[0], imgs.size(0))

        optim.zero_grad()
        loss.backward()
        optim.step()

        if iteration % args.print_freq == 0:
            log_str = 'Train: [{0}/{1}/{top1.count:}]\tepoch: {epoch:}\titer: {iteration:}\t' \
                  'Loss: {loss.val:.4f} ({loss.avg:.4f})\t' \
                  'Prec@1: {top1.val:.3f} ({top1.avg:.3f})\t' \
                  'Prec@5: {top5.val:.3f} ({top5.avg:.3f})\tlr {lr:.6f}'.format(
                batch_idx, len(train_loader), epoch=epoch, iteration=iteration,
                lr=optim.param_groups[0]['lr'],
                batch_time=batch_time, loss=losses, top1=top1, top5=top5)
            print(log_str)

        if lr_scheduler is not None:
            lr_scheduler.step()  # update lr


    log_str = 'Train_summary: [{0}/{1}/{top1.count:}]\tepoch: {epoch:}\titer: {iteration:}\t' \
                  'Time: {batch_time.avg:.3f}\t' \
                  'Loss: {loss.avg:.4f}\tPrec@1: {top1.avg:.3f}\tPrec@5: {top5.avg:.3f}\tlr {lr:.6f}'.format(
                batch_idx, len(train_loader), epoch=epoch, iteration=iteration,
                lr=optim.param_groups[0]['lr'],
                batch_time=batch_time, loss=losses, top1=top1, top5=top5)
    print(log_str)
    gc.collect()


Train epoch=0, iter=0:  10%|██                   | 2/20 [00:00<00:08,  2.09it/s]

Train: [0/20/40]	epoch: 0	iter: 0	Loss: 0.0000 (0.0000)	Prec@1: 100.000 (100.000)	Prec@5: 100.000 (100.000)	lr 0.100000


Train_summary: [19/20/781]	epoch: 0	iter: 19	Time: 0.000	Loss: 0.3056	Prec@1: 99.872	Prec@5: 100.000	lr 0.100000


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

In [14]:
torch.save(net.state_dict(), 'finetuned2.pth')

In [15]:
validate(training=True)

Test_summary: [17/18/175] iter: 18	BestPrec@1: 100.000	BestPrec@5: 100.000	Loss: 0.0000	Prec@1: 100.000	Prec@5: 100.000	
